In [8]:
import ee
import numpy as np
import matplotlib.pyplot as plt

ee.Initialize()
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

In [9]:
# Define date range
start_date = ee.Date('2018-01-01')
end_date = start_date.advance(180, 'day')    # 180 days later
sentinel1 = sentinel1.filterDate(start_date, end_date)

# Grab the VH and VV polarized parts in interferometric
# wide swath mode
vh = sentinel1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(
    ee.Filter.eq('instrumentMode', 'IW'))

# Separate data collected when the sat was ascending and descending
ascending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));
descending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

# Create a composite from means at different polarizations
# and look angles *for visualization*
composite = ee.Image.cat([
    ascending.select('VH').max(),
    descending.select('VH').max(), 
    ee.ImageCollection(ascending.select('VV').merge(
        descending.select('VV'))).max()]).focal_median();

In [10]:
llx = 49
lly = 24
urx = 52
ury = 27
geometry = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]

task_config = {
    'scale': 200,  
    'region': geometry
    }

task = ee.batch.Export.image(composite, 'vhExport', task_config)
task.start()